In [5]:
#"Advanced Learning Algorithms" Final Task

import numpy as np
import matplotlib.pyplot as plt
from utils import *

In [6]:
#We have a list of mushrooms. Features are: Cap color (brown or read), stalk shape (tapering or enlarging), solitary, edible
# Hot coding by giving brown = 1, tapering = 1, solitary = 1, edible = 1

X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]]) #encoding features
y_train = np.array([1,1,0,0,1,0,0,1,1,0]) #edible or poisonous (output)

In [25]:
#Steps for building a decision tree:

#1. Calculate the entropy at a node
#2. Split the dataset at a node into left and right branches based on a given feature
#3. Calculate the information gain from splitting on a given feature
#4. Choose the feature that maximizes information gain
#5. Use helper functions to build a decision tree by repeating splitting process until stopping criteria is met (here, max depth = 2)

#Step 1. Compute entropy

def compute_entropy(y):
    """
    Computes the entropy for

    Args:
       y (ndarray): Numpy array indicating whether each example at a node is
           edible (`1`) or poisonous (`0`)

    Returns:
        entropy (float): Entropy at that node

    """
    entropy = 0.

    if len(y) == 0:
        return 0.0

    p_1 = np.sum(y == 1) / len(y) #compute fraction that are edible

     # For p1 = 0 and 1, set the entropy to 0 (to handle 0log0)
    if p_1 == 0 or p_1 == 1:
        return 0
    else:
        entropy = -p_1 * np.log2(p_1) - (1 - p_1) * np.log2(1 - p_1)

    return entropy

#Step 2: Split dataset
# Splitting dataset into left ad right branches

def split_dataset(X, node_indices, feature):

  left_indices = []
  right_indices = []

  for i in node_indices:
        if X[i][feature] == 1:
            left_indices.append(i)
        else:
            right_indices.append(i)

  return left_indices, right_indices

  #Step 3: Calculate information gain

  def compute_information_gain(X, y, node_indices, feature):

    left_indices, right_indices = split_dataset(X, node_indices, feature)

    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]

    information_gain = 0

    node_entropy = compute_entropy(y_node) # entropy at node
    left_entropy = compute_entropy(y_left) # entropy after branching
    right_entropy = compute_entropy(y_right)

    w_left = len(X_left)/len(X_node)
    w_right = len(X_right)/len(X_node) #finding weighted fractions

    weighted_entropy = w_left * left_entropy + w_right * right_entropy

    information_gain = node_entropy - weighted_entropy
    #info gain is change in entropy after split compared to before (want to reduce)


    return information_gain

    #Step 4: Using info gain to get best split

  def get_best_split(X, y, node_indices):

    max_information_gain=0
    for feature in range(num_features):
        information_gain = compute_information_gain(X, y, node_indices, feature)
        if information_gain > max_information_gain:
            max_information_gain = information_gain
            best_feature = feature

    return best_feature

    #Step 4: Building the tree

    # Not graded
tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):

    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return

    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature = get_best_split(X, y, node_indices)

    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))

    # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    tree.append((left_indices, right_indices, best_feature))

    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)
